The main objective of this analysis is to come up with a regression model that can predict car sales price by training a model in the given data set also highlighting the main features that affect the price of the car. 

Data set description: 
The dataset contains car sales prices with the details of a description of each item and the sales price  
The dataset was downloaded from kaggle  
The data set is a 19237 rows x 18 columns with the flowing columns  
ID: the id of the car 
Price: price of the car (Target Column) 
Levy 
Manufacturer: name of the manufacturer 
Model: name of model 
Prod. Year: year of production  
Category: the category of the car (Sedan, Jeep, Minivan, etc…) 
Leather interior: dose it have it or not  
Fuel type 
Engine volume: in liters  
Mileage: in kilometers  Cylinders: number of cylinders  Gear box type: auto ,manual; ,etc.  
Drive wheels: 4*4, front , rear  
Doors: number of doors 
Wheel: number of wheels  
Color 
Airbags: number of air bags 


First importing the necessary libraries  

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from scipy import stats
import os
import numpy as np
import pandas as pd
from scipy.stats import boxcox
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import keras 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
path='/kaggle/input/car-price-prediction-challenge/car_price_prediction.csv'
df=pd.read_csv(path ,sep='\t')
data=pd.read_csv(path ,)
data.drop(labels='ID', axis=1)
data.head()
data.columns
data.head()

Inspecting the dataset and dealing with duplicates 

In [ ]:
data.shape
data1=data

In [ ]:
data.info()


In [ ]:
data.duplicated().sum()
data= data.drop_duplicates()


Dealing with columns with numeric values but implemented as objects

In [ ]:
dt_obj = data.select_dtypes(include = ['object'])
col=dt_obj
mil=[]
lev=[]
engvol=[]
for stri in dt_obj['Mileage'] :
   mil.append(int(stri.strip('km')))
for stri in dt_obj['Engine volume'] :
   engvol.append(float(stri.strip('Turbo')))
for stri in dt_obj['Levy'] :
   lev.append(int(stri.replace('-','0')))
data.drop('Mileage', inplace=True, axis=1)
data.insert(7, "Mileage",mil)
data.drop('Levy', inplace=True, axis=1)
data.insert(7, "Levy",lev)       
data.drop('Engine volume', inplace=True, axis=1)
data.insert(8, "Engine volume",engvol)      
data
dt_obj = data.select_dtypes(include = ['object'])




In [ ]:
data1=data

Plotting the price against other numeric values ,this doesn’t look comprehendible due to outliers  

In [ ]:
dt_num = data1.select_dtypes(include = ['float64', 'int64'])
dt_num_corr = dt_num.corr()['Price'][:-1]
for i in range(0, len(dt_num.columns), 10):
    sns.pairplot(data=dt_num,
                x_vars=dt_num.columns[i:i+10],
                y_vars=['Price'])

dt_num.boxplot()


dealing with outliers 

In [ ]:

dt_num.boxplot('Price')

In [ ]:
data1.sort_values(by = 'Price', ascending = False)[:20]


In [ ]:
data.sort_values(by = 'Price', ascending = False
                )
data1= data1.drop(data.index[[16983,8541,1225]])

In [ ]:
sns.boxplot(x=data1['Price'])
data1.sort_values(by = 'Price', ascending = False)[:10]
data1 = data1.drop(data1[(data1['Price'] >308906)].index)


In [ ]:
sns.boxplot(x=data['Prod. year'])
data.sort_values(by = 'Prod. year', ascending = True)[:10]
data1.describe()



In [ ]:

sns.boxplot(x=data1['Levy'])


In [ ]:
data.sort_values(by = 'Levy', ascending = False)[:10]
data1= data1.drop(data1[(data1['Levy'] >10000)].index)

In [ ]:
data.sort_values(by = 'Mileage', ascending = False)[:10]
#data1= data1.drop(data.index[[15347,11901,15393,2278,19167,6157,12734,17582,19199,]])
data1.sort_values(by = 'Mileage', ascending = False)[:20]
data1 = data1.drop(data1[(data1['Mileage'] >23000000)].index)
data1.sort_values(by = 'Mileage', ascending = False)[:10]
data1 = data1.drop(data1[(data1['Price'] <800)].index)
data1 = data1.drop(data1[(data1['Price'] >308906)].index)


In [ ]:
sns.boxplot(x=data1['Cylinders'])


In [ ]:
sns.boxplot(x=data1['Airbags'])


In [ ]:
sns.boxplot(x=data1['Engine volume'])
data1 = data1.drop(data1[(data1['Engine volume'] ==20.0)].index)
data1.sort_values(by = 'Engine volume', ascending = False)[:10]






In [ ]:
data1.groupby('Price').mean().sort_values(by = 'Price', ascending = False)


Plotting the price against other numeric values after removing the outliers  


In [ ]:
import matplotlib.pyplot as plt
dt_num = data1.select_dtypes(include = ['float64', 'int64'])

for i in range(0, len(dt_num.columns), 10):
    sns.pairplot(data=dt_num,
                x_vars=dt_num.columns[i:i+10],
                y_vars=['Price'])

checking correlation between the price and numreic values

In [ ]:
dt_num = data1.select_dtypes(include = ['float64', 'int64'])
dt_num_corr = dt_num.corr()['Price'][:-1] # -1 means that the latest row is SalePrice
top_features = dt_num_corr[abs(dt_num_corr) > 0.2].sort_values(ascending=False) #displays pearsons correlation coefficient greater than 0.5
print("There is {}  correlated values with SalePrice:\n{}".format(len(top_features), top_features))

Pair plotting all the numeric values against each other

In [ ]:
sns.pairplot(dt_num)

checking skewness and dealing with it

In [ ]:
print("Skewness: %f" % data['Price'].skew())
skevals=dt_num.skew()

In [ ]:
skew_limit = 0.75 # define a limit above which we will log transform

skew_cols = (skevals
             
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

skew_cols




In [ ]:
data1['Prod. year']=data1['Prod. year']-min(data1['Prod. year'])




In [ ]:

for col,skew in zip(dt_num.columns,skevals):
  if   col== "ID"or col== "Prod. year": 
      continue 
  data1[col]+=1
  data1[col] , l= boxcox(data1[col])
  if   col== "Price":
   print(l)






 

  

  


In [ ]:
data1['Doors']

In [ ]:

data1= data1.drop_duplicates()


In [ ]:
data1.groupby('Drive wheels').mean()

In [ ]:
dt_num = data1.select_dtypes(include = ['float64', 'int64'])
dt_num_corr = dt_num.corr()['Price'][:-1] # -1 means that the latest row is SalePrice
top_features = dt_num_corr[abs(dt_num_corr) > 0.1].sort_values(ascending=False) #displays pearsons correlation coefficient greater than 0.5
print("There is {} strongly correlated values with SalePrice:\n{}".format(len(top_features), top_features))

dt_num

converting the doors column from date to object 

In [ ]:
data1['Doors'] = data1['Doors'].replace('04-May', '4-5')
data1['Doors'] = data1['Doors'].replace('02-Mar','2-3')

one hot encoding object columns

In [ ]:
from sklearn.preprocessing import LabelEncoder 
labelencoder = LabelEncoder()
data1['Doors'] = labelencoder.fit_transform(data1['Doors'])


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

mask = data.dtypes == object
categorical_cols = data.columns[mask]
categorical_cols

data2=data1[categorical_cols]
data2






In [ ]:
data1[categorical_cols]=data2[categorical_cols]
data3=data1[categorical_cols]
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohc = OneHotEncoder( drop='first',)
le = LabelEncoder()
data3col=data3.columns

for col in data3col:
    
    # Integer encode the string categories
    dat = le.fit_transform(data3[col]).astype(np.int)
    
    # Remove the original column from the dataframe
    data3 = data3.drop(col, axis=1)

    # One hot encode the data--this returns a sparse array
    new_dat = ohc.fit_transform(dat.reshape(-1,1))

    # Create unique column names
    n_cols = new_dat.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]

    # Create the new dataframe
    new_df = pd.DataFrame(new_dat.toarray(), 
                          index=data3.index, 
                          columns=col_names)

    # Append the new data to the dataframe
    data3 = pd.concat([data3, new_df], axis=1)
data3

In [ ]:
data2=data1
data2 = data2.drop(categorical_cols, axis=1)
data2 = pd.concat([data2, data3], axis=1)
data2 = data2.drop("ID", axis=1)
dt_num = dt_num.drop("ID", axis=1)
data2



train ,test data split

In [ ]:
from sklearn.model_selection import train_test_split
y_col = 'Price'
X_data=[]
y_data=[]
feature_cols = [x for x in data2.columns if x != y_col]
X_data = data2[feature_cols]
y_data = data2[y_col]
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    test_size=0.3, random_state=42)
data1
#s = StandardScaler()
#s.fit(X_train)
#X_train=s.transform(X_train)
#X_test=s.transform(X_test)




X_train



In [ ]:
X_train2

A pipeline of standard scalar , polynomial features, and a linear regressor 

In [ ]:
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
pf = PolynomialFeatures(degree=2)
X_train2=X_train
X_test2=X_test
pf.fit(X_train2)
X_train2=pf.transform(X_train2)
X_test2=pf.transform(X_test2)
LR = LinearRegression()
LR2 = LinearRegression()



Fitting predicting and calculating the errors

In [ ]:
LR2 = LR2.fit(X_train2, y_train)



y_train_pred2 = LR2.predict(X_train2)

y_test_pred2 = LR2.predict(X_test2)

In [ ]:


error_df2=list()

error_df2.append(pd.Series({'train': mean_squared_error(y_train, y_train_pred2),
                           'train2':r2_score(y_train, y_train_pred2),
                           'test' : mean_squared_error(y_test, y_test_pred2),
                          'test2' : r2_score(y_test, y_test_pred2)},
                          
                          ))
error_df2



extracting the Coefficients and sorting them

In [ ]:
coefficients = pd.DataFrame()
p = PolynomialFeatures(degree=2).fit(data2[feature_cols])
coefficients['linear regression'] = LR2.coef_.ravel()
coefficients.applymap(abs)


coefficients['columns']=p.get_feature_names(data2[feature_cols].columns)



coefficients.sort_values(by = 'linear regression', ascending = False)[:20]


Plotting the predictions vs. the ground truth

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
LR = LinearRegression()
error_df = list()

print('done')
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
pf = PolynomialFeatures(degree=2)

LR = LR.fit(X_train, y_train)



y_train_pred = LR.predict(X_train)

y_test_pred = LR.predict(X_test)
#predictions = cross_val_predict(LR, X_train, y_train, cv = kf)

error_df.append(pd.Series({'train': mean_squared_error(y_train, y_train_pred),
                           'test' : mean_squared_error(y_test,  y_test_pred)},
                           name='no enc'))
error_df
r2_score(y_test, y_test_pred)
pf = PolynomialFeatures(degree=2)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
alphas2 = np.geomspace(20, 1000, 20)
scores2=[]
s=StandardScaler()


estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("lr", LR)])
estimator.get_params().keys()
r2_score(y_test, y_test_pred)




In [ ]:

coefficients2 = pd.DataFrame()


coefficients2['linear regression'] = LR.coef_.ravel()
coefficients2.applymap(abs)
coefficients2['columns']=data2[feature_cols].columns



coefficients2.sort_values(by = 'linear regression', ascending = False)[:20]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


sns.set_context('talk')
sns.set_style('ticks')
sns.set_palette('dark')

ax = plt.axes()
# we are going to use y_test, y_test_pred
ax.scatter(y_test, y_test_pred, alpha=.5)
ax.plot(y_test, y_test, alpha=.5)
ax.plot(y_test, y_test, alpha=1)
ax.set(xlabel='Ground truth', 
       ylabel='Predictions')


In [ ]:
LR = LinearRegression()
pf = PolynomialFeatures()



estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("lr", LR)])
estimator.get_params().keys()


In [ ]:
params = {
    'polynomial_features__degree': [1,2],
    'polynomial_features__include_bias':[True,False],
    
}

gridLR = GridSearchCV(estimator, params, cv=kf,verbose=10)
gridLR.fit(X_train,y_train)
predictions=gridLR.predict(X_test)



In [ ]:
scoresLR=[]
predictions=gridLR.predict(X_test)

scoresLR.append(r2_score(y_test, predictions))
scoresLR.append(mean_squared_error(y_test, predictions))


scoresLR

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


sns.set_context('talk')
sns.set_style('ticks')
sns.set_palette('dark')

ax = plt.axes()
# we are going to use y_test, y_test_pred
ax.scatter(y_train, y_train_pred, alpha=.5)
ax.plot(y_train, y_train, alpha=.5)
ax.plot(y_train, y_train, alpha=1)
ax.set(xlabel='Ground truth', 
       ylabel='Predictions')




A ridge regression model was fitted using a grid search with a scalar, polynomial degree of 1 or 2, an alpha of A geometrical space in range of 20 and 1000 and 20 values. The grid search also used a k fold cross validation with 3 folds using the same random state and train test split as before. The plot of alphas vs. scores  

In [ ]:
pf = PolynomialFeatures(degree=2)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
alphas2 = np.geomspace(20, 1000, 20)
ridge = Ridge()
scores2=[]


estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("ridge_regression", ridge)])
estimator.get_params().keys()

In [ ]:
estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("ridge_regression", ridge)])
params = {
    'polynomial_features__degree': [1,2],
    'polynomial_features__include_bias':[True,False],
    'ridge_regression__alpha': alphas2,
}
gridd = GridSearchCV(estimator, params, cv=kf,verbose=10)
gridd.fit(X_train,y_train)

 

getting the best estimator and plotting 

In [ ]:
gridd.best_estimator_

In [ ]:

ax = plt.axes()
#y_train_pred=gridd.predict(X_train)
#pred=gridd.predict(X_test)
# we are going to use y_test, y_test_pred
ax.scatter(y_test, pred2, alpha=.5)
ax.plot(y_test, y_test, alpha=.5)
ax.plot(y_test, y_test, alpha=1)
ax.set(xlabel='Ground truth', 
       ylabel='Predictions')

A lasso regressor was only trained in first degree polynomial due to the slowness of convergence the  
Using k-fold cross validation with alpha in range of [0 – 10 ]and the best performing  model had an alpha of alpha=0.0215 


In [ ]:
pf = PolynomialFeatures(degree=1)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
alphas2 = np.geomspace(0.0000001, 10, 10)
scores2=[]
lass = Lasso(max_iter=100000)


estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("lasso", lass)] )
estimator.get_params().keys()

score

In [ ]:
scorelass=[]

scorelass.append( r2_score(y_train, predictions))

scorelass.append(mean_squared_error(y_train, predictions))
scorelass
gridlass.best_estimator_
lass = Lasso(alpha=0.021544346900318822, max_iter=100000)


In [ ]:
score2 = r2_score(y_train, predictions)

scores2.append(score2)
score2 = mean_squared_error(y_train, predictions)
scores2.append(score2)
scores2
np.geomspace(2000, 2500, 5)

In [ ]:
params = {
    'lasso__alpha': alphas2,
}
scorelass=[]

gridlass = GridSearchCV(estimator, params, cv=kf,verbose=10)
gridlass.fit(X_train,y_train)
predictions=gridlass.predict(X_train)


alphas vs score


In [ ]:
pf = PolynomialFeatures(degree=2)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
alphas = np.geomspace(2000, 2500, 5)
scores=[]
for alpha in alphas:
    ridge = Ridge(alpha=alpha, max_iter=100000)

    estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("ridge_regression", ridge)])

    predictions = cross_val_predict(estimator, X_train, y_train, cv = kf)
    score = r2_score(y_train, predictions)
    scores.append(score)

    
plt.plot(alphas, scores)
grid3 = GridSearchCV(estimator, params, cv=kf,verbose=10)

In [ ]:
Lasso = Lasso(alpha=0.021544346900318822, max_iter=100000)
pf = PolynomialFeatures(degree=1)
s.fit(X_train)
x1=s.transform(X_train)
x2=s.transform(X_test)
pf.fit(X_train)
x1=pf.transform(x1)
x2=pf.transform(x2)
Lasso.fit(x1,y_train)


In [ ]:
pred1=Lasso.predict(x1)
pred2=Lasso.predict(x2)

error_df3=list()
error_df3.append(pd.Series({'train mean squared error': mean_squared_error(y_train, pred1),
                           'train r2 scoore':r2_score(y_train, pred1),
                           'test mean squared error' : mean_squared_error(y_test, pred2),
                          'test r2 score' : r2_score(y_test, pred2)},
                          
                          ))
error_df3

In [ ]:
coefficients4= pd.DataFrame()


coefficients4['LASSO regression coefficients'] = Lasso.coef_.ravel()
coefficients4.applymap(abs)
p = PolynomialFeatures(degree=1).fit(data2[feature_cols])

coefficients4['columns']= p.get_feature_names(data2[feature_cols].columns)



coefficients4.sort_values(by = 'LASSO regression coefficients', ascending = False)[:20]

In [ ]:
pf = PolynomialFeatures(degree=1)
kf = KFold(shuffle=True, random_state=72018, n_splits=3)
alphas = np.geomspace(0.0000001,10 , 1)
scores=[]
for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=100000)

    estimator = Pipeline([
        ("scaler", s),
        ("polynomial_features", pf),
        ("ridge_regression", lasso)])

    predictions = cross_val_predict(estimator, X_train, y_train, cv = kf)
    score = r2_score(y_train, predictions)
    scores.append(score)

    
plt.plot(alphas, scores)
#grid4 = GridSearchCV(estimator, params, cv=kf,verbose=10)


predicition vs ground reality

In [ ]:
score

ax = plt.axes()
ax.scatter(y_train, predictions)
    
